In [ ]:
#MT on Total Dead Kilometers Opportunity Matrix (TOCM)
import pandas as pd
import numpy as np
import random

from collections import defaultdict
randomlist = []
df_demand = {}
random.seed(20) # Use the same random seed
for i in range(0,25):   # 20 depots
    n = random.randint(100,150) # Depot Capacity
    randomlist.append(n)

for i in range(25):
    df_demand[i] = randomlist[i]

sum = 0
for (k,v) in df_demand.items():
    sum = sum + v
    
df_supply = {}
for i in range(sum):
    df_supply[i] = 1.0
    
np.random.seed(20) # Use the same random seed
df_dkm = pd.DataFrame(np.random.uniform(0.5,40, size=(sum,25))).astype(float)

random.seed(20) # Use the same random seed
age_b = []
for i in range(sum):
    n = random.randint(1,3) #Only to generate integer values inclusive of 1 & 3
    age_b.append(n)
    
age = {}
for i in range(len(age_b)):
    age[i] = age_b[i] 
    
kpl = []
co2 = []
random.seed(20) # Use the same random seed
for (k,v) in age.items():
    if (v==1):
        n = random.uniform(5.1,6)
        m = round(random.uniform(515, 524), 2)
        kpl.append(n)
        co2.append(m)
    if (v==2):
        n = random.uniform(4.1,5)
        m = round(random.uniform(525, 534), 2)
        kpl.append(n)
        co2.append(m)
    if (v==3):
        n = random.uniform(3.1,4)
        m = round(random.uniform(535, 540), 2)
        kpl.append(n)
        co2.append(m)
        
fc_kpl = {}
for i in range(sum):
    fc_kpl[i] =(111/ kpl[i])

co2_c = {}
for i in range(sum):
    co2_c[i] =(0.00118642*co2[i])
    
random.seed(20) # Use the same random seed
doc = {}
for i in range(0,25):
    n = random.uniform(50,100)
    doc[i] = n
    
df_tdkom_c = pd.DataFrame(np.zeros(df_dkm.shape))
for i in range(len(df_dkm.columns)):
    for j in range(len(df_dkm.index)):
        df_tdkom_c[i][j] = df_dkm[i][j]*fc_kpl[j] + df_dkm[i][j]*co2_c[j] + doc[i]
#df_tdkom_c

In [20]:
df_cost = df_tdkom_c.copy(deep=True)
# Calculating the row deadkm opportunity matrix 
df_rowmin = df_cost.min(axis=1)
df_rowopp = df_cost.copy(deep=True)
for i in range(0,len(df_cost)):
    for j in range(0,len(df_cost.columns)):
        df_rowopp.iloc[i,j] = df_rowopp.iloc[i,j] - df_rowmin.iloc[i]

# Calculating the column deadkm opportunity matrix 
df_colmin = df_cost.min(axis=0)
df_colopp = df_cost.copy(deep=True)
for i in range(0,len(df_cost.columns)):
    for j in range(0,len(df_cost)):
        df_colopp.iloc[j,i] = df_colopp.iloc[j,i] - df_colmin.iloc[i]

# Calculating the total deadkm opportunity matrix 
df_tdkom = round(df_rowopp + df_colopp, 2)
#print("Total Deadkm opportunity matrix is:")
#print(df_tdkom)


df_res = df_tdkom.copy(deep=True)
df_result = pd.DataFrame(np.zeros(df_tdkom.shape))

In [21]:
# Ranking each depot depending on value of DK
df_rank= (df_tdkom.rank(1, ascending=True, method='first')).astype(int)     

df_depot = df_demand.copy()

R = 1
df_result = pd.DataFrame(np.zeros(df_tdkom.shape))

In [22]:
while(R!=len(df_depot)+1): 
    df_rankr = (df_rank == R)*1.0

    lst = []
    for m in df_rankr.columns:
        if (df_rankr.sum(axis=0)[m] <= df_demand[m]):
            for j in df_rankr.index:
                df_result.loc[j,m] = df_rankr.loc[j,m] 

    for m in df_rankr.columns:     
        if (df_rankr.sum(axis=0)[m] > df_demand[m]):
                for j in df_rankr.index:
                    if (df_rank.loc[j,m] == R):
                        lst.append((j,m))

                new = []
                for (k,v) in lst:
                    if (v == m):
                        new.append(df_tdkom.loc[k,v])      

                new.sort()
                n = int(df_demand[m])
                newn = new[0:n]

                for j in df_rankr.index:
                    for i in range(len(newn)):
                        if (newn[i] == df_tdkom.loc[j,m]): 
                            df_result.loc[j,m] = df_rankr.loc[j,m]  

    for m in df_rankr.columns:
        df_demand[m] = df_depot[m] - df_result.sum(axis=0)[m] 

    for m in df_rankr.columns: 
        if (df_demand[m] == 0):
                df_rank = df_rank.drop(m, axis=1)
                del df_demand[m]

    for k in df_rankr.index:
        df_supply[k] = df_supply[k] - df_result.sum(axis=1)[k]

    for i in df_rankr.index:
        if (df_supply[i] == 0):
            df_rank = df_rank.drop(i, axis = 0)
            del df_supply[i]

    R = R + 1

In [23]:
df_result

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2961,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2962,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2963,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2964,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [24]:
df_final = df_cost.mul(df_result, axis = 0)
df_final.to_numpy().sum()

402641.389089049